In [96]:
# Import necessary libraries.
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kstest
import numpy as np
import scipy.stats as spstats
from scipy.stats import shapiro
from sklearn.preprocessing import *
import warnings
from sklearn import linear_model
from skater.core.explanations import Interpretation
from skater.model import InMemoryModel
from skater.core.local_interpretation.lime.lime_tabular import LimeTabularExplainer
from IPython.display import display, HTML, clear_output

from progressbar import *

import data_clean
import feature_engineering
import feature_filtering
import step_by_step
import evaluation

reload(data_clean)
plt.style.use('ggplot')
%matplotlib inline
warnings.filterwarnings('ignore')

In [3]:
# Read in data for test.
features = pd.read_csv('data/pokemon.csv')
combats = pd.read_csv('data/combats.csv')
test_data = pd.read_csv('data/tests.csv')

In [55]:
# Explore data.
features[0:3]

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False


In [4]:
column_name = list(features.columns + '_1') + list(features.columns + '_2') + ['win']
results = pd.DataFrame(columns=column_name)

In [ ]:
progress = ProgressBar()
for num in progress(xrange(combats.shape[0])):
    # Fetch the features of each player.
    player_1 = features[features['#']==combats['First_pokemon'].loc[num]]
    player_2 = features[features['#'] == combats['Second_pokemon'].loc[num]]
    # Fetch result of combat.
    if combats['First_pokemon'].loc[num] == combats['Winner'].loc[num]:
        win = [1]
    else:
        win = [0]
    # Assign vector combination to result dataframe.
    results.loc[num] = np.array(player_1).tolist()[0] + np.array(player_2).tolist()[0] + list(win)

In [6]:
results[0:3]

,#_1,Name_1,Type 1_1,Type 2_1,HP_1,Attack_1,Defense_1,Sp. Atk_1,Sp. Def_1,Speed_1,...,Type 2_2,HP_2,Attack_2,Defense_2,Sp. Atk_2,Sp. Def_2,Speed_2,Generation_2,Legendary_2,win
0,266.0,Larvitar,Rock,Ground,50.0,64.0,50.0,45.0,50.0,41.0,...,Dark,70.0,70.0,40.0,60.0,40.0,60.0,3.0,False,0.0
1,702.0,Virizion,Grass,Fighting,91.0,90.0,72.0,90.0,129.0,108.0,...,Fighting,91.0,129.0,90.0,72.0,90.0,108.0,5.0,True,0.0
2,191.0,Togetic,Fairy,Flying,55.0,40.0,85.0,80.0,105.0,40.0,...,NaN,75.0,75.0,75.0,125.0,95.0,40.0,5.0,False,0.0


In [10]:
# Explore data.
combats[0:3]

,First_pokemon,Second_pokemon,Winner
0,266,298,298
1,702,701,701
2,191,668,668


In [11]:
# Explore data.
test_data[0:3]

,First_pokemon,Second_pokemon
0,129,117
1,660,211
2,706,115


In [97]:
target_column = 'win'
id_columns = ['#_1', 'Name_1', '#_2', 'Name_2']
result_1 = data_clean.wash_data(results, target_column, id_columns, outlier=False)

AttributeError: 'CategoryFeatureEngineer' object has no attribute 'cate_columns'

In [76]:
column = 'Defense_1'
results[column][results[column].isnull().values].index.tolist()[0]

IndexError: list index out of range

In [54]:
data = results
data[column + '_flag'] = np.zeros(shape=len(data[column]))

In [55]:
data[column + '_flag'][np.where(results[column].isnull().values == True)[0].tolist()] = 1

In [65]:
data[column] = data[[column]]

In [85]:
test = data.columns.tolist()


In [86]:
test.remove('Name_1')

In [87]:
test

['#_1',
 'Type 1_1',
 'Type 2_1',
 'HP_1',
 'Attack_1',
 'Defense_1',
 'Sp. Atk_1',
 'Sp. Def_1',
 'Speed_1',
 'Generation_1',
 'Legendary_1',
 '#_2',
 'Name_2',
 'Type 1_2',
 'Type 2_2',
 'HP_2',
 'Attack_2',
 'Defense_2',
 'Sp. Atk_2',
 'Sp. Def_2',
 'Speed_2',
 'Generation_2',
 'Legendary_2',
 'win',
 'Defense_1_flag']